In [1]:
from customtkinter import *
from PIL import Image, ImageTk

In [2]:
class SmallFrame(CTkFrame):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        self.algo_label = CTkLabel(self, text="Algorithm:", fg_color="transparent")
        self.algo_label.grid(row=0, column=0, padx=5, pady=0)
        
        self.combobox = CTkComboBox(self, values=["Trial Divisions", "Solovay-Strassen", "Miller", "Miller-Rabin"], command=self.combobox_callback)
        self.combobox.set("Miller-Rabin")
        self.combobox.grid(row=0, column=1, padx=5, pady=0)
        
    def combobox_callback(choice):
        print("Combobox dropdown clicked:", choice)
        
class PalettesFrame(CTkFrame):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        self.bright_theme_button = CTkButton(self, text="", command=self.bright_theme_button_click, width=26, height=26, fg_color="white")
        self.bright_theme_button.grid(row=0, column=0, padx=5, pady=0)

        self.dark_theme_button = CTkButton(self, text="", command=self.dark_theme_button_click, width=26, height=26, fg_color="black")
        self.dark_theme_button.grid(row=0, column=1, padx=5, pady=0)
    
    def bright_theme_button_click(self):
        print("Theme button click")
        
    def dark_theme_button_click(self):
        print("Theme button click")

class HeaderFrame(CTkFrame):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        
        self.small_frame = SmallFrame(master=self)
        self.small_frame.grid(row=0, column=0, padx=20, pady=10)
        
        self.palletes_frame = PalettesFrame(master=self)
        self.palletes_frame.grid(row=0, column=1, padx=68, pady=10)
        
        info_ico = CTkImage(Image.open("assets/info.png"), size=(16, 16))
        self.info_button = CTkButton(self, text="", command=self.info_button_click, width=26, height=26, image=info_ico)
        self.info_button.grid(row=0, column=2, padx=40, pady=10)
        
    def info_button_click(self):
        print("Info button click")

class MainFrame(CTkFrame):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)
        self.grid_columnconfigure(1, weight=1)
        
        self.entry_for_num_to_check = CTkEntry(self, placeholder_text="Enter the number...", width=426)
        self.entry_for_num_to_check.grid(row=0, column=0, padx=20, pady=20)
        
        self.check_button = CTkButton(self, text="Check", command=self.check_button_click, width=72)
        self.check_button.grid(row=0, column=1, padx=20, pady=20)
        
    def check_button_click(self):
        print("Check button click")

class App(CTk):
    def __init__(self):
        super().__init__()
        set_appearance_mode("dark")
        self.geometry("600x600")
        self.title("Prime checker")
        # self.grid_rowconfigure(0, weight=0)
        # self.grid_columnconfigure(0, weight=0)
        # self.grid_rowconfigure(1, weight=0)

        self.header = HeaderFrame(master=self, width=580, height=30)
        self.header.grid(row=0, column=0, padx=10, pady=5)
        
        self.main_frame = MainFrame(master=self)
        self.main_frame.grid(row=1, column=0, padx=10, pady=0)

app = App()
app.iconbitmap('assets\\favicon.ico')
app.resizable(False,False)
app.mainloop()